In [ ]:
pip install ultralytics opencv-python-headless

  Using cached ultralytics-8.3.167-py3-none-any.whl.metadata (37 kB)
  Using cached ultralytics_thop-2.0.14-py3-none-any.whl.metadata (9.4 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.4.5.8-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.2.1.3-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.5.147-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.6.1.9-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.3.1.170-py3-none-manylinux2014_x86_64.whl.

In [ ]:
# main.py
# Import necessary libraries
import cv2
from ultralytics import YOLO
import os

# This import is specific to Google Colab for file handling
try:
    from google.colab import files
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def object_detection_video(input_path, output_path, confidence_threshold=0.5):
    """
    Performs object detection on a video file using a pre-trained YOLOv8 model
    and saves the result to a new video file.

    Args:
        input_path (str): The path to the input video file.
        output_path (str): The path where the output video will be saved.
        confidence_threshold (float): The minimum confidence score for a detection.
    """
    try:
        # --- 1. Load a pre-trained YOLO model ---
        print("Loading YOLOv8 model...")
        model = YOLO('yolov8n.pt')
        print("Model loaded successfully.")

        # --- 2. Open Video File ---
        print(f"Opening video file: {input_path}")
        cap = cv2.VideoCapture(input_path)
        if not cap.isOpened():
            print(f"Error: Could not open video file '{input_path}'. Please ensure it's a valid video file.")
            return

        # --- 3. Get Video Properties and Setup Writer ---
        frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(cap.get(cv2.CAP_PROP_FPS))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        # Define the codec and create VideoWriter object
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

        print(f"Video properties: {frame_width}x{frame_height} @ {fps} FPS")
        print(f"Output will be saved to: {output_path}")

        # --- 4. Process Video Stream ---
        frame_count = 0
        while True:
            ret, frame = cap.read()
            if not ret:
                break # End of video

            frame_count += 1
            print(f"Processing frame {frame_count}/{total_frames}...")

            # Perform prediction on the frame
            results = model.predict(source=frame, save=False, verbose=False)
            result = results[0]
            class_names = result.names

            # Iterate over each detected object
            for box in result.boxes:
                confidence = box.conf[0]
                if confidence > confidence_threshold:
                    coords = box.xyxy[0].tolist()
                    x1, y1, x2, y2 = map(int, coords)
                    class_id = int(box.cls[0])
                    class_name = class_names[class_id]
                    label = f"{class_name}: {confidence:.2f}"

                    # Draw bounding box and label
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

            # Write the processed frame to the output file
            out.write(frame)

    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        # --- 5. Release Resources ---
        if 'cap' in locals() and cap.isOpened():
            cap.release()
        if 'out' in locals() and out.isOpened():
            out.release()
        print(f"Processing complete. Video saved to {output_path}")
        # If in Colab, offer to download the result
        if IN_COLAB:
            try:
                files.download(output_path)
            except Exception as e:
                print(f"Could not automatically download file. Please find it in the file browser. Error: {e}")


if __name__ == "__main__":
    if IN_COLAB:
        print("Please upload a video file.")
        # Prompt user to upload a file
        uploaded = files.upload()

        # Check if a file was uploaded
        if not uploaded:
            print("\nNo file was uploaded. Please run the cell again and select a file.")
        else:
            # Get the filename of the uploaded file
            input_video_file = next(iter(uploaded))
            print(f"\nSuccessfully uploaded: {input_video_file}")

            # Define the output filename
            output_video_file = "result_" + os.path.basename(input_video_file)

            # Run the object detection on the uploaded video
            object_detection_video(input_video_file, output_video_file)
    else:
        print("This script is configured for Google Colab file uploads.")
        print("To run locally, please modify the '__main__' block to point to a local video file.")
        # Example for local execution:
        # input_video_file = "path/to/your/video.mp4"
        # output_video_file = "local_result.mp4"
        # if os.path.exists(input_video_file):
        #     object_detection_video(input_video_file, output_video_file)



Please upload a video file.


Saving WhatsApp Video 2025-07-17 at 14.28.40_12097d87.mp4 to WhatsApp Video 2025-07-17 at 14.28.40_12097d87.mp4

Successfully uploaded: WhatsApp Video 2025-07-17 at 14.28.40_12097d87.mp4
Loading YOLOv8 model...
Model loaded successfully.
Opening video file: WhatsApp Video 2025-07-17 at 14.28.40_12097d87.mp4
Video properties: 640x360 @ 30 FPS
Output will be saved to: result_WhatsApp Video 2025-07-17 at 14.28.40_12097d87.mp4
Processing frame 1/150...
Processing frame 2/150...
Processing frame 3/150...
Processing frame 4/150...
Processing frame 5/150...
Processing frame 6/150...
Processing frame 7/150...
Processing frame 8/150...
Processing frame 9/150...
Processing frame 10/150...
Processing frame 11/150...
Processing frame 12/150...
Processing frame 13/150...
Processing frame 14/150...
Processing frame 15/150...
Processing frame 16/150...
Processing frame 17/150...
Processing frame 18/150...
Processing frame 19/150...
Processing frame 20/150...
Processing frame 21/150...
Processing fram

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>